In [222]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim

In [223]:
df = pd.read_csv("Data/LeagueofLegends.csv")
df.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


In [224]:
df.shape

(7620, 57)

In [225]:
df['Season'].unique()

array(['Spring', 'Summer'], dtype=object)

In [226]:
df['Type'].unique()

array(['Season', 'Playoffs', 'Regional', 'International', 'Promotion'],
      dtype=object)

In [227]:
df['League'].value_counts()

LCK      1445
NALCS    1272
EULCS    1099
LMS       778
TCL       653
OPL       458
WC        308
CBLoL     301
LCL       281
LJL       258
LLN       242
CLS       175
IEM       138
MSI       111
RR        101
Name: League, dtype: int64

In [228]:
df.columns

Index(['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult', 'rResult',
       'redTeamTag', 'gamelength', 'golddiff', 'goldblue', 'bKills', 'bTowers',
       'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldred', 'rKills',
       'rTowers', 'rInhibs', 'rDragons', 'rBarons', 'rHeralds', 'blueTop',
       'blueTopChamp', 'goldblueTop', 'blueJungle', 'blueJungleChamp',
       'goldblueJungle', 'blueMiddle', 'blueMiddleChamp', 'goldblueMiddle',
       'blueADC', 'blueADCChamp', 'goldblueADC', 'blueSupport',
       'blueSupportChamp', 'goldblueSupport', 'blueBans', 'redTop',
       'redTopChamp', 'goldredTop', 'redJungle', 'redJungleChamp',
       'goldredJungle', 'redMiddle', 'redMiddleChamp', 'goldredMiddle',
       'redADC', 'redADCChamp', 'goldredADC', 'redSupport', 'redSupportChamp',
       'goldredSupport', 'redBans', 'Address'],
      dtype='object')

In [229]:
def new_df_gold_length(df):
    """
    Crée un nouveau dataframe avec comme colonnes 'League', 'Type','blueTeamTag','bResult','rResult',
    'redTeamTag','gamelength' et 'golddiff'.

    Args:
        df: dataframe initial

    Returns:
        Nouveau dataframe.
    """
    
    # NALCS : Amérique du Nord
    # LCK : Corée
    # EULCS : Europe
    # LMS : Taiwan Hong-Kong Macao
    # TCL : Turquie
    # OPL : Océanie
    # WC : World Cup
    # CBLoL : Brésil
    
    df2 = df[['League', 'Type','blueTeamTag','bResult','rResult','redTeamTag','gamelength','golddiff']]

    df2=df2[(df2['League'] == 'NALCS') | 
            (df2['League'] == 'LCK')   | 
            (df2['League'] == 'EULCS') |
            (df2['League'] == 'TCL')   |
            (df2['League'] == 'CBLoL') ]
            
    df2= df2[(df2['Type'] != 'Regional') & 
             (df2['Type'] != 'Promotion')]

    return df2

In [230]:
df2 = new_df_gold_length(df)
df2.head()

,League,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff
0,NALCS,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,NALCS,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,NALCS,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,NALCS,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,NALCS,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [231]:
df2.shape

(4315, 8)

In [232]:
df2['League'].unique()

array(['NALCS', 'EULCS', 'LCK', 'CBLoL', 'TCL'], dtype=object)

In [233]:
df2['Type'].unique()

array(['Season', 'Playoffs'], dtype=object)

In [234]:
df2.dtypes

League         object
Type           object
blueTeamTag    object
bResult         int64
rResult         int64
redTeamTag     object
gamelength      int64
golddiff       object
dtype: object

In [235]:
df2["golddiff"][0][0]

'['

In [236]:
def gold_to_list(df):
    """
    Modifie la colonne "golddiff" en passant ses valeurs de string à list

    Args:
        df : dataframe où mettre la colonne
    """
    new_gold_diff = []
    for elements in df["golddiff"]:
        i=1
        s=''
        tab=[]
        num = 0
        for i in range (1,len(elements)):
            if elements[i] == ',' or elements[i] == ']':
                num = pd.to_numeric(s)
                s='' 
                tab.append(num)   
            else :
                s=s+elements[i]            
            i=i+1
        new_gold_diff.append(tab)
    position = df.columns.get_loc("golddiff")
    del df["golddiff"]
    df.insert(position, "golddiff", new_gold_diff)

In [237]:
gold_to_list(df2)
df2.head()

,League,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff
0,NALCS,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,NALCS,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,NALCS,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,NALCS,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,NALCS,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [238]:
df2.dtypes

League         object
Type           object
blueTeamTag    object
bResult         int64
rResult         int64
redTeamTag     object
gamelength      int64
golddiff       object
dtype: object

In [239]:
df2["golddiff"][0][0]

0

In [240]:
location = app.geocode("North America").raw
            
print(location["lat"])
print(location["lon"])

51.0000002
-109


In [241]:
print(app.geocode("North America").raw["lat"])

51.0000002


In [242]:
def add_localisation(df, position):
    """
    Ajoute une colonne "Country", "Lattitude" et "Longitude" au dataframe mis en argument selon la ligue du match en question.

    Args:
        df : dataframe où ajouter les colonnes
        position : numéro de la colonne à partir de laquelle ajouter les colonnes
    """
    
    app = Nominatim(user_agent="Adam")

    loc_k  = app.geocode("Korea").raw
    loc_na = app.geocode("North America").raw
    loc_eu = app.geocode("Europe").raw
    loc_br = app.geocode("Brazil").raw
    loc_t  = app.geocode("Turkey").raw

    k  = [loc_k["lat"] , loc_k["lon"]]
    na = [loc_na["lat"], loc_na["lon"]]
    eu = [loc_eu["lat"], loc_eu["lon"]]
    br = [loc_br["lat"], loc_br["lon"]]
    t  = [loc_t["lat"] , loc_t["lon"]]

    country = []
    lattitude = []
    longitude = []

    for elements in df["League"] :
        if elements == "LCK" :
            country.append("Korea")
            lattitude.append(k[0])
            longitude.append(k[1])
        elif elements == "NALCS" :
            country.append("North America")
            lattitude.append(na[0])
            longitude.append(na[1])
        elif elements == "EULCS" :
            country.append("Europe")
            lattitude.append(eu[0])
            longitude.append(eu[1])
        elif elements == "CBLoL" :
            country.append("Brazil")
            lattitude.append(br[0])
            longitude.append(br[1])
        elif elements == "TCL" :
            country.append("Turkey")
            lattitude.append(t[0])
            longitude.append(t[1])
        else :
            country.append("not found")
            lattitude.append("not found")
            longitude.append("not found")
            
    df.insert(position, "Country", country)
    df.insert(position+1, "Lattitude", lattitude)
    df.insert(position+2, "Longitude", longitude)
    df["Lattitude"]  = df["Lattitude"].astype("float")
    df2["Longitude"] = df["Longitude"].astype("float")

In [243]:
add_localisation(df2,1)
df2.head()

,League,Country,Lattitude,Longitude,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff
0,NALCS,North America,51.0,-109.0,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,NALCS,North America,51.0,-109.0,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,NALCS,North America,51.0,-109.0,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,NALCS,North America,51.0,-109.0,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,NALCS,North America,51.0,-109.0,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [244]:
df2.dtypes

League          object
Country         object
Lattitude      float64
Longitude      float64
Type            object
blueTeamTag     object
bResult          int64
rResult          int64
redTeamTag      object
gamelength       int64
golddiff        object
dtype: object